In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [78]:
pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [98]:
import pickle

with open("/content/drive/MyDrive/collectionandqueries/BM25", 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    bm25 = pickle.load(f)

In [4]:
pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 7.4 MB/s 
     |████████████████████████████████| 88 kB 9.3 MB/s 
     |████████████████████████████████| 662 kB 67.1 MB/s 
     |████████████████████████████████| 5.2 MB 64.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 6.3 MB/s 
     |████████████████████████████████| 5.8 MB 71.1 MB/s 
     |████████████████████████████████| 1.3 MB 50.3 MB/s 
     |████████████████████████████████| 182 kB 76.5 MB/s 
     |████████████████████████████████| 7.6 MB 69.5 MB/s 
     |████████████████████████████████| 1.1 MB 65.6 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700860 sha256=7ba25d0a6e92cc01256e720bc84664dfd009ae5bbfc33dac01dedca5ef9b6c43
  Stored in directory: /root/.cache/

In [99]:
from bertopic import BERTopic
BERT_model_FINAL = BERTopic.load("/content/drive/MyDrive/collectionandqueries/20KDocs/qrels_BERT_model_FINAL_preprocessed")

In [101]:
from math import log, sqrt
ALPHA = 1
BETA = 0.75
GAMMA = 0.15

def generateInvertedIndex(invertedIndex, doc):
        line = doc
        text = line.strip().split()
        for word in text:
            if word not in invertedIndex.keys():
                invertedIndex[word] = 1
            else:
                invertedIndex[word] += 1

def queryFrequency(query):
    queryFreq = {}
    for term in query.split():
        if term in queryFreq.keys():
            queryFreq[term] += 1
        else:
            queryFreq[term] = 1
    return queryFreq

def calculateDocsCount(doc, docIndex):
    line = doc
    text = line.strip().split()
    for term in text:
      if term in docIndex.keys():
        docIndex[term] += 1
      else:
        docIndex[term] = 1

def findDocs(all_docs):
    relIndex = {}
    k = len(all_docs)
    for i in range(0, k):
      doc = all_docs[i]
      calculateDocsCount(doc, relIndex)
    return relIndex

def findRelDocMagnitude(docIndex):
    mag = 0
    for term in docIndex:
        mag += float(docIndex[term]**2)
        mag = float(sqrt(mag))
    return mag

def findRocchioScore(term, queryFreq, relDocMag, relIndex):
    if(term not in queryFreq.keys()):
      queryFreq[term] = 0
    if(term not in relIndex.keys()):
      relIndex[term] = 0  
    rocchioScore = ALPHA * queryFreq[term] + (BETA/relDocMag) * relIndex[term] 
    return rocchioScore


def findNewQuery(query, all_docs):
    invertedIndex = {}
    for doc in all_docs:
      generateInvertedIndex(invertedIndex, doc) 
    queryFreq = queryFrequency(query)
    relIndex = findDocs(all_docs)
    relDocMag = findRelDocMagnitude(relIndex)
    updatedQuery = {}
    newQuery = query
    for term in invertedIndex.keys():
        updatedQuery[term] = findRocchioScore(term, queryFreq, relDocMag, relIndex)
    sortedUpdatedQuery = sorted(updatedQuery.items(), key=lambda x:x[1], reverse=True)
    if len(sortedUpdatedQuery)<5:
        loopRange = len(sortedUpdatedQuery)
    else:
        loopRange = 5
    for i in range(loopRange):
        term,frequency = sortedUpdatedQuery[i]
        if term not in query:
            newQuery +=  " "
            newQuery +=  term
    return newQuery

In [ ]:
import csv
import sys

csv.field_size_limit(sys.maxsize)


fr6 = open("/content/drive/MyDrive/collectionandqueries/all_query_ids.tsv", "r")
fr7 = open("/content/drive/MyDrive/collectionandqueries/all_queries.tsv", "r")
fr8 = open("/content/drive/MyDrive/collectionandqueries/all_document_ids.tsv", "r")
fr9 = open("/content/drive/MyDrive/collectionandqueries/all_documents.tsv", "r")
fr10 = open("/content/drive/MyDrive/collectionandqueries/all_labels.tsv", "r")

input_file1 = csv.reader(fr1, delimiter = "\t")
input_file2 = csv.reader(fr2, delimiter = "\t")
input_file3 = csv.reader(fr3, delimiter = "\t")
input_file4 = csv.reader(fr4, delimiter = "\t")
input_file5 = csv.reader(fr5, delimiter = "\t")

all_query_ids2 = []
all_queries2 = []
all_document_ids2 = []
all_documents2 = []
all_labels2 = []

for w in input_file1: 
    #print('w : ' + str(w[0]))
    all_query_ids2.append(w[0])

for w in input_file2: 
    all_queries2.append(w[0])

for w in input_file3: 
    all_document_ids2.append(w[0])

for w in input_file4: 
    all_documents2.append(w[0])

for w in input_file5: 
    all_labels2.append(w[0]) 

print('len(all_query_ids2) : ' + str(len(all_query_ids2)))
print('len(all_queries2) : ' + str(len(all_queries2)))
print('len(all_document_ids2) : ' + str(len(all_document_ids2)))
print('len(all_documents2) : ' + str(len(all_documents2)))
print('len(all_labels2) : ' + str(len(all_labels2)))

all_query_ids = []
all_queries = []
all_document_ids = []
all_documents = []
all_labels = []

length = len(all_query_ids2)
index = 0

for index in range(0, length): 
    #print('index : ' + str(index))
    all_query_ids.append(all_query_ids2[index])
    all_queries.append(all_queries2[index])
    all_document_ids.append(all_document_ids2[index])
    all_documents.append(all_documents2[index])
    all_labels.append(int(all_labels2[index]))
    """
    print('all_query_ids2[index] : ' + all_query_ids2[index])
    print('all_queries2[index] : ' + all_queries2[index])
    print('all_document_ids2[index] : ' + all_document_ids2[index])
    print('all_documents2[index] : ' + all_documents2[index])
    print('all_labels2[index] : ' + all_labels2[index])
    """
    
    if(int(all_labels2[index])) == 1:
      cur_query = all_queries2[index]
      print('cur_query : ' + cur_query)
      topics_res, similarity = BERT_model_FINAL.find_topics(cur_query, top_n=2) 
      for j in range(0, 2):
        all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[j])
        newQuery = findNewQuery(cur_query, all_topic_docs)
        print('newQuery : ' + newQuery)
        all_query_ids.append(all_query_ids2[index])
        all_queries.append(newQuery)
        all_document_ids.append(all_document_ids2[index])
        all_documents.append(all_documents2[index])
        all_labels.append(int(all_labels2[index]))

print('len(all_query_ids) : ' + str(len(all_query_ids)))
print('len(all_queries) : ' + str(len(all_queries)))
print('len(all_document_ids) : ' + str(len(all_document_ids)))
print('len(all_documents) : ' + str(len(all_documents)))
print('len(all_labels) : ' + str(len(all_labels)))

In [ ]:
import os

pos_neg_ration = 4

# Maximal number of training samples we want to use
max_train_samples = 5e4


### Now we read the MS Marco dataset
data_folder = '/content/drive/MyDrive/collectionandqueries/'
os.makedirs(data_folder, exist_ok=True)


#### Read the corpus files, that contain all the passages. Store them in the corpus dict
corpus = {}
collection_filepath = os.path.join(data_folder, 'collection.tsv')
if not os.path.exists(collection_filepath):
    print('collection.tsv not found')

with open(collection_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        corpus[pid] = passage
print('len(corpus.keys() : ' + str(len(corpus.keys())))

### Read the train queries, store in queries dict
queries = {}
queries_filepath = os.path.join(data_folder, 'queries.train.tsv')
if not os.path.exists(queries_filepath):
    print('queries.train.tsv not found')

with open(queries_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[qid] = query

print('len(queries.keys() : ' + str(len(queries.keys())))

### Now we create our training & dev data
original_query_ids = []
original_queries = []
original_document_ids = [] 
original_labels = []
original_documents = []

"""
train_samples = []
dev_samples = {}

# We use 200 random queries from the train set for evaluation during training
# Each query has at least one relevant and up to 200 irrelevant (negative) passages
num_dev_queries = 200
num_max_dev_negatives = 200

# msmarco-qidpidtriples.rnd-shuf.train-eval.tsv.gz and msmarco-qidpidtriples.rnd-shuf.train.tsv.gz is a randomly
# shuffled version of qidpidtriples.train.full.2.tsv.gz from the MS Marco website
# We extracted in the train-eval split 500 random queries that can be used for evaluation during training
train_eval_filepath = os.path.join(data_folder, 'msmarco-qidpidtriples.rnd-shuf.train-eval.tsv')
if not os.path.exists(train_eval_filepath):
    print('msmarco-qidpidtriples.rnd-shuf.train-eval.tsv not found')

with open(train_eval_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, pos_id, neg_id = line.strip().split()

        if qid not in dev_samples and len(dev_samples) < num_dev_queries:
            dev_samples[qid] = {'query': queries[qid], 'positive': set(), 'negative': set()}

        if qid in dev_samples:
            dev_samples[qid]['positive'].add(corpus[pos_id])
            all_query_ids.append(qid)
            all_queries.append(queries[qid])
            all_document_ids.append(pos_id)
            all_documents.append(corpus([pos_id]))
            all_labels.append(1)
            

            if len(dev_samples[qid]['negative']) < num_max_dev_negatives:
                dev_samples[qid]['negative'].add(corpus[neg_id])
                all_query_ids.append(qid)
                all_queries.append(queries[qid])
                all_document_ids.append(neg_id)
                all_documents.append(corpus([neg_id]))
                all_labels.append(0)
                

print('len(dev_samples.keys()) : ' + str(len(dev_samples.keys())))
"""

# Read our training file
train_filepath = os.path.join(data_folder, 'qidpidtriples-subset.train-200K.tsv')
if not os.path.exists(train_filepath):
    print('qidpidtriples-subset.train-200K.tsv not found')

cnt = 0
with open(train_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, pos_id, neg_id = line.strip().split()

        query = queries[qid]
        if (cnt % (pos_neg_ration+1)) == 0:
            passage = corpus[pos_id]
            label = 1
            original_query_ids.append(qid)
            original_queries.append(query)
            original_document_ids.append(pos_id)
            original_documents.append(passage)
            original_labels.append(label)
        else:
            passage = corpus[neg_id]
            label = 0
            original_query_ids.append(qid)
            original_queries.append(queries[qid])
            original_document_ids.append(neg_id)
            original_documents.append(passage)
            original_labels.append(label)

        #train_samples.append(InputExample(texts=[query, passage], label=label))
        cnt += 1

        if cnt >= max_train_samples:
            break
print('len(original_query_ids) : ' + str(len(original_query_ids)))
print('len(original_queries) : ' + str(len(original_queries)))
print('len(original_document_ids) : ' + str(len(original_document_ids)))
print('len(original_documents) : ' + str(len(original_documents)))
print('len(original_labels) : ' + str(len(original_labels)))

len(corpus.keys() : 8841823
len(queries.keys() : 808731
len(original_query_ids) : 50000
len(original_queries) : 50000
len(original_document_ids) : 50000
len(original_documents) : 50000
len(original_labels) : 50000


In [ ]:
import os

pos_neg_ration = 4

# Maximal number of training samples we want to use
#max_train_samples = 5e4
max_train_samples = 5e3

### Now we read the MS Marco dataset
data_folder = '/content/drive/MyDrive/collectionandqueries/'
os.makedirs(data_folder, exist_ok=True)


#### Read the corpus files, that contain all the passages. Store them in the corpus dict
corpus = {}
collection_filepath = os.path.join(data_folder, 'collection.tsv')
if not os.path.exists(collection_filepath):
    print('collection.tsv not found')

with open(collection_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        corpus[pid] = passage
print('len(corpus.keys() : ' + str(len(corpus.keys())))

### Read the train queries, store in queries dict
queries = {}
queries_filepath = os.path.join(data_folder, 'queries.train.tsv')
if not os.path.exists(queries_filepath):
    print('queries.train.tsv not found')

with open(queries_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[qid] = query

print('len(queries.keys() : ' + str(len(queries.keys())))

### Now we create our training & dev data
all_query_ids = []
all_queries = []
all_document_ids = [] 
all_labels = []
all_documents = []

"""
train_samples = []
dev_samples = {}

# We use 200 random queries from the train set for evaluation during training
# Each query has at least one relevant and up to 200 irrelevant (negative) passages
num_dev_queries = 200
num_max_dev_negatives = 200

# msmarco-qidpidtriples.rnd-shuf.train-eval.tsv.gz and msmarco-qidpidtriples.rnd-shuf.train.tsv.gz is a randomly
# shuffled version of qidpidtriples.train.full.2.tsv.gz from the MS Marco website
# We extracted in the train-eval split 500 random queries that can be used for evaluation during training
train_eval_filepath = os.path.join(data_folder, 'msmarco-qidpidtriples.rnd-shuf.train-eval.tsv')
if not os.path.exists(train_eval_filepath):
    print('msmarco-qidpidtriples.rnd-shuf.train-eval.tsv not found')

with open(train_eval_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, pos_id, neg_id = line.strip().split()

        if qid not in dev_samples and len(dev_samples) < num_dev_queries:
            dev_samples[qid] = {'query': queries[qid], 'positive': set(), 'negative': set()}

        if qid in dev_samples:
            dev_samples[qid]['positive'].add(corpus[pos_id])
            all_query_ids.append(qid)
            all_queries.append(queries[qid])
            all_document_ids.append(pos_id)
            all_documents.append(corpus([pos_id]))
            all_labels.append(1)
            

            if len(dev_samples[qid]['negative']) < num_max_dev_negatives:
                dev_samples[qid]['negative'].add(corpus[neg_id])
                all_query_ids.append(qid)
                all_queries.append(queries[qid])
                all_document_ids.append(neg_id)
                all_documents.append(corpus([neg_id]))
                all_labels.append(0)
                

print('len(dev_samples.keys()) : ' + str(len(dev_samples.keys())))
"""

# Read our training file
train_filepath = os.path.join(data_folder, 'qidpidtriples-subset.train-200K.tsv')
if not os.path.exists(train_filepath):
    print('qidpidtriples-subset.train-200K.tsv not found')

cnt = 0
with open(train_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, pos_id, neg_id = line.strip().split()
        print('cnt : ' + str(cnt))
        query = queries[qid]
        if (cnt % (pos_neg_ration+1)) == 0:
            #print('positive')
            passage = corpus[pos_id]
            label = 1
            all_query_ids.append(qid)
            all_queries.append(query)
            all_document_ids.append(pos_id)
            all_documents.append(passage)
            all_labels.append(label)
            bm25_passages = bm25.get_top_n(query, original_documents, n=3)
            for bm25_passage in bm25_passages:
              all_query_ids.append(qid)
              all_queries.append(query)
              all_document_ids.append(pos_id)
              all_documents.append(bm25_passage)
              all_labels.append(label)
        else:
            #print('negative')
            passage = corpus[neg_id]
            label = 0
            all_query_ids.append(qid)
            all_queries.append(queries[qid])
            all_document_ids.append(neg_id)
            all_documents.append(passage)
            all_labels.append(label)

        #train_samples.append(InputExample(texts=[query, passage], label=label))
        cnt += 1

        if cnt >= max_train_samples:
            break
print('len(all_query_ids) : ' + str(len(all_query_ids)))
print('len(all_queries) : ' + str(len(all_queries)))
print('len(all_document_ids) : ' + str(len(all_document_ids)))
print('len(all_documents) : ' + str(len(all_documents)))
print('len(all_labels) : ' + str(len(all_labels)))

In [1]:
import csv
import sys

csv.field_size_limit(sys.maxsize)


fr1 = open("/content/drive/MyDrive/collectionandqueries/all_query_ids.tsv", "r")
fr2 = open("/content/drive/MyDrive/collectionandqueries/all_queries.tsv", "r")
fr3 = open("/content/drive/MyDrive/collectionandqueries/all_document_ids.tsv", "r")
fr4 = open("/content/drive/MyDrive/collectionandqueries/all_documents.tsv", "r")
fr5 = open("/content/drive/MyDrive/collectionandqueries/all_labels.tsv", "r")

input_file1 = csv.reader(fr1, delimiter = "\t")
input_file2 = csv.reader(fr2, delimiter = "\t")
input_file3 = csv.reader(fr3, delimiter = "\t")
input_file4 = csv.reader(fr4, delimiter = "\t")
input_file5 = csv.reader(fr5, delimiter = "\t")

all_query_ids2 = []
all_queries2 = []
all_document_ids2 = []
all_documents2 = []
all_labels2 = []

for w in input_file1: 
    #print('w : ' + str(w[0]))
    all_query_ids2.append(w[0])

for w in input_file2: 
    all_queries2.append(w[0])

for w in input_file3: 
    all_document_ids2.append(w[0])

for w in input_file4: 
    all_documents2.append(w[0])

for w in input_file5: 
    all_labels2.append(w[0]) 

print('len(all_query_ids2) : ' + str(len(all_query_ids2)))
print('len(all_queries2) : ' + str(len(all_queries2)))
print('len(all_document_ids2) : ' + str(len(all_document_ids2)))
print('len(all_documents2) : ' + str(len(all_documents2)))
print('len(all_labels2) : ' + str(len(all_labels2)))

all_query_ids = []
all_queries = []
all_document_ids = []
all_documents = []
all_labels = []

length = len(all_query_ids2)
index = 0

for index in range(0, length): 
    #print('index : ' + str(index))
    all_query_ids.append(all_query_ids2[index])
    all_queries.append(all_queries2[index])
    all_document_ids.append(all_document_ids2[index])
    all_documents.append(all_documents2[index])
    all_labels.append(int(all_labels2[index]))
    """
    print('all_query_ids2[index] : ' + all_query_ids2[index])
    print('all_queries2[index] : ' + all_queries2[index])
    print('all_document_ids2[index] : ' + all_document_ids2[index])
    print('all_documents2[index] : ' + all_documents2[index])
    print('all_labels2[index] : ' + all_labels2[index])
    """
    
    if(int(all_labels2[index])) == 1:
      cur_query = all_queries2[index]
      #print('cur_query : ' + cur_query)
      topics_res, similarity = BERT_model_FINAL.find_topics(cur_query, top_n=2) 
      for j in range(0, 2):
        all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[j])
        newQuery = findNewQuery(cur_query, all_topic_docs)
        #print('newQuery : ' + newQuery)
        all_query_ids.append(all_query_ids2[index])
        all_queries.append(newQuery)
        all_document_ids.append(all_document_ids2[index])
        all_documents.append(all_documents2[index])
        all_labels.append(int(all_labels2[index]))

print('len(all_query_ids) : ' + str(len(all_query_ids)))
print('len(all_queries) : ' + str(len(all_queries)))
print('len(all_document_ids) : ' + str(len(all_document_ids)))
print('len(all_documents) : ' + str(len(all_documents)))
print('len(all_labels) : ' + str(len(all_labels)))

FileNotFoundError: ignored

In [84]:
from sklearn.model_selection import train_test_split
train_queries, val_queries, train_docs, val_docs, train_labels, val_labels = train_test_split(
    all_queries, 
    all_documents, 
    all_labels, 
    test_size=.2
)

In [85]:
!pip install pandas transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [86]:
from transformers import BertTokenizerFast

model_name = "google/bert_uncased_L-4_H-512_A-8"
tokenizer = BertTokenizerFast.from_pretrained(model_name)

train_encodings = tokenizer(train_queries, train_docs, truncation=True, padding='max_length', max_length=128)
val_encodings = tokenizer(val_queries, val_docs, truncation=True, padding='max_length', max_length=128)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--google--bert_uncased_L-4_H-512_A-8/snapshots/606e4d55252882ac25ba1f1d1a182075830f5a90/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--bert_uncased_L-4_H-512_A-8/snapshots/606e4d55252882ac25ba1f1d1a182075830f5a90/config.json
Model config BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-512_A-8",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_toke

In [87]:
import torch

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = OurDataset(train_encodings, train_labels)
val_dataset = OurDataset(val_encodings, val_labels)


In [88]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(model_name)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--bert_uncased_L-4_H-512_A-8/snapshots/606e4d55252882ac25ba1f1d1a182075830f5a90/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--google--bert_uncased_L-4_H-512_A-8/snapshots/606e4d55252882ac25ba1f1d1a182075830f5a90/pytorch_model.bin
Some weights of the model checkpoint at google/bert_uncased_L-4_H-512_A-8 were not

In [90]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [91]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
pip install trectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import ndcg_score
from sklearn.metrics import classification_report
import numpy as np
import evaluate
import tensorflow as tf


"""
def my_compute_metrics_ndcg(eval_pred):
    predictions, labels = eval_pred
    print ('predictions : ' + str(predictions))
    predictions2 = tf.nn.log_softmax(predictions, axis=-1)
    #print(classification_report(labels, predictions))
    return ndcg_score(predictions2, labels)
"""

def my_compute_metrics_precision(eval_pred):
    predictions, labels = eval_pred
    metric = evaluate.load("precision")
    #print('predictions : ' + str(predictions))
    #print('labels : ' + str(labels))
    #if task != "stsb":
    predictions = np.argmax(predictions, axis=-1)
    #else:
    #predictions = predictions[:]
    print(classification_report(labels, predictions))
    return metric.compute(predictions=predictions, references=labels)


# You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
# predictions and label_ids field) and has to return a dictionary string to float.
def compute_metrics2(pred):
    """
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)   
    """
    print('pred : ' + str(pred))
    # Get the metric function
    trec_eval = evaluate.load("trec_eval")
    qrel = {}
    qids, seen, i = [], {}, 0
    for q in val_queries:
      if q in seen:
        qids.append(seen[q])
      else:
        seen[q] = i
        qids.append(i)
        i+=1
    qrel={
      "query": qids,
      "q0": val_queries,
      "docid": val_docs,
      "rel": val_labels
    }
    run = {
      "query": qids,
      "q0": val_queries,
      "docid": val_docs,
      "rank": pred[1].tolist(),
      "score": [max(l) for l in pred[0].tolist()],
      "system": ["test" for i in qids]
    }    
    results = trec_eval.compute(references=[qrel], predictions=[run]) 
    return results

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=0.005,
    save_total_limit=1,              # limit the total amount of checkpoints. Deletes the older checkpoints.    
)


trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics = compute_metrics2
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 12800
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1600
  Number of trainable parameters = 1026


Epoch,Training Loss,Validation Loss,Runid,Num Ret,Num Rel,Num Rel Ret,Num Q,Map,Gm Map,Bpref,Rprec,Recip Rank,P@5,P@10,P@15,P@20,P@30,P@100,P@200,P@500,P@1000,Ndcg@5,Ndcg@10,Ndcg@15,Ndcg@20,Ndcg@30,Ndcg@100,Ndcg@200,Ndcg@500,Ndcg@1000
1,0.379200,0.279177,test,3200,2389,2389,2574,0.686092,nan,0.001166,0.686092,0.686092,0.185625,0.092813,0.061875,0.046406,0.030938,0.009281,0.004641,0.001856,0.000928,0.685805,0.685805,0.685805,0.685805,0.685805,0.685805,0.685805,0.685805,0.685805
2,0.309300,0.241448,test,3200,2389,2389,2574,0.685897,nan,0.000777,0.685703,0.685897,0.185625,0.092813,0.061875,0.046406,0.030938,0.009281,0.004641,0.001856,0.000928,0.685805,0.685805,0.685805,0.685805,0.685805,0.685805,0.685805,0.685805,0.685805


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3200
  Batch size = 64


pred : <transformers.trainer_utils.EvalPrediction object at 0x7fb0ee5d1c10>


Trainer is attempting to log a value of "test" of type <class 'str'> for key "eval/runid" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3200
  Batch size = 64


pred : <transformers.trainer_utils.EvalPrediction object at 0x7fb0e98b38e0>


Trainer is attempting to log a value of "test" of type <class 'str'> for key "eval/runid" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1600, training_loss=0.3467003762722015, metrics={'train_runtime': 2059.9403, 'train_samples_per_second': 12.428, 'train_steps_per_second': 0.777, 'total_flos': 253117897113600.0, 'train_loss': 0.3467003762722015, 'epoch': 2.0})

In [62]:
import csv
import sys

csv.field_size_limit(sys.maxsize)


fr1 = open("/content/drive/MyDrive/collectionandqueries/all_query_ids.tsv", "r")
fr2 = open("/content/drive/MyDrive/collectionandqueries/all_queries.tsv", "r")
fr3 = open("/content/drive/MyDrive/collectionandqueries/all_document_ids.tsv", "r")
fr4 = open("/content/drive/MyDrive/collectionandqueries/all_documents.tsv", "r")
fr5 = open("/content/drive/MyDrive/collectionandqueries/all_labels.tsv", "r")

input_file1 = csv.reader(fr1, delimiter = "\t")
input_file2 = csv.reader(fr2, delimiter = "\t")
input_file3 = csv.reader(fr3, delimiter = "\t")
input_file4 = csv.reader(fr4, delimiter = "\t")
input_file5 = csv.reader(fr5, delimiter = "\t")

all_query_ids2 = []
all_queries2 = []
all_document_ids2 = []
all_documents2 = []
all_labels2 = []

for w in input_file1: 
    #print('w : ' + str(w[0]))
    all_query_ids2.append(w[0])

for w in input_file2: 
    all_queries2.append(w[0])

for w in input_file3: 
    all_document_ids2.append(w[0])

for w in input_file4: 
    all_documents2.append(w[0])

for w in input_file5: 
    all_labels2.append(w[0]) 

print('len(all_query_ids2) : ' + str(len(all_query_ids2)))
print('len(all_queries2) : ' + str(len(all_queries2)))
print('len(all_document_ids2) : ' + str(len(all_document_ids2)))
print('len(all_documents2) : ' + str(len(all_documents2)))
print('len(all_labels2) : ' + str(len(all_labels2)))

all_query_ids = []
all_queries = []
all_document_ids = []
all_documents = []
all_labels = []

length = len(all_query_ids2)
index = 0

for index in range(0, length): 
    #print('index : ' + str(index))
    all_query_ids.append(all_query_ids2[index])
    all_queries.append(all_queries2[index])
    all_document_ids.append(all_document_ids2[index])
    all_documents.append(all_documents2[index])
    all_labels.append(int(all_labels2[index]))
    """
    print('all_query_ids2[index] : ' + all_query_ids2[index])
    print('all_queries2[index] : ' + all_queries2[index])
    print('all_document_ids2[index] : ' + all_document_ids2[index])
    print('all_documents2[index] : ' + all_documents2[index])
    print('all_labels2[index] : ' + all_labels2[index])
    """
    
    if(int(all_labels2[index])) == 1:
      cur_query = all_queries2[index]
      #print('cur_query : ' + cur_query)
      topics_res, similarity = BERT_model_FINAL.find_topics(cur_query, top_n=2) 
      for j in range(0, 2):
        all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[j])
        newQuery = findNewQuery(cur_query, all_topic_docs)
        #print('newQuery : ' + newQuery)
        all_query_ids.append(all_query_ids2[index])
        all_queries.append(cur_query)
        all_document_ids.append(all_document_ids2[index])
        all_documents.append(all_documents2[index])
        all_labels.append(int(all_labels2[index]))

print('len(all_query_ids) : ' + str(len(all_query_ids)))
print('len(all_queries) : ' + str(len(all_queries)))
print('len(all_document_ids) : ' + str(len(all_document_ids)))
print('len(all_documents) : ' + str(len(all_documents)))
print('len(all_labels) : ' + str(len(all_labels)))

len(all_query_ids2) : 8000
len(all_queries2) : 8000
len(all_document_ids2) : 8000
len(all_documents2) : 8000
len(all_labels2) : 8000
len(all_query_ids) : 16000
len(all_queries) : 16000
len(all_document_ids) : 16000
len(all_documents) : 16000
len(all_labels) : 16000


In [73]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import ndcg_score
from sklearn.metrics import classification_report
import numpy as np
import evaluate
import tensorflow as tf


"""
def my_compute_metrics_ndcg(eval_pred):
    predictions, labels = eval_pred
    print ('predictions : ' + str(predictions))
    predictions2 = tf.nn.log_softmax(predictions, axis=-1)
    #print(classification_report(labels, predictions))
    return ndcg_score(predictions2, labels)
"""

def my_compute_metrics_precision(eval_pred):
    predictions, labels = eval_pred
    metric = evaluate.load("precision")
    #print('predictions : ' + str(predictions))
    #print('labels : ' + str(labels))
    #if task != "stsb":
    predictions = np.argmax(predictions, axis=-1)
    #else:
    #predictions = predictions[:]
    print(classification_report(labels, predictions))
    return metric.compute(predictions=predictions, references=labels)


# You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
# predictions and label_ids field) and has to return a dictionary string to float.
def compute_metrics2(pred):
    """
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)   
    """
    print('pred : ' + str(pred))
    # Get the metric function
    trec_eval = evaluate.load("trec_eval")
    qrel = {}
    qids, seen, i = [], {}, 0
    for q in val_queries:
      if q in seen:
        qids.append(seen[q])
      else:
        seen[q] = i
        qids.append(i)
        i+=1
    qrel={
      "query": qids,
      "q0": val_queries,
      "docid": val_docs,
      "rel": val_labels
    }
    run = {
      "query": qids,
      "q0": val_queries,
      "docid": val_docs,
      "rank": pred[1].tolist(),
      "score": [max(l) for l in pred[0].tolist()],
      "system": ["test" for i in qids]
    }    
    results = trec_eval.compute(references=[qrel], predictions=[run]) 
    return results

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=0.005,
    save_total_limit=1,              # limit the total amount of checkpoints. Deletes the older checkpoints.    
)


trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics = compute_metrics2
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 12800
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1600
  Number of trainable parameters = 1026


Epoch,Training Loss,Validation Loss,Runid,Num Ret,Num Rel,Num Rel Ret,Num Q,Map,Gm Map,Bpref,Rprec,Recip Rank,P@5,P@10,P@15,P@20,P@30,P@100,P@200,P@500,P@1000,Ndcg@5,Ndcg@10,Ndcg@15,Ndcg@20,Ndcg@30,Ndcg@100,Ndcg@200,Ndcg@500,Ndcg@1000
1,0.411900,0.324054,test,3200,2381,3285,1743,0.869092,nan,0.001147,0.676946,0.531268,0.321170,0.185198,0.125301,0.094234,0.062823,0.018847,0.009423,0.003769,0.001885,0.675070,0.675278,0.675278,0.675278,0.675278,0.675278,0.675278,0.675278,0.675278
2,0.349800,0.288268,test,3200,2381,3285,1743,0.868447,nan,0.001147,0.676946,0.531268,0.321170,0.185198,0.125301,0.094234,0.062823,0.018847,0.009423,0.003769,0.001885,0.675070,0.675278,0.675278,0.675278,0.675278,0.675278,0.675278,0.675278,0.675278


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-1500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3200
  Batch size = 64


pred : <transformers.trainer_utils.EvalPrediction object at 0x7fb0edbfbaf0>


Trainer is attempting to log a value of "test" of type <class 'str'> for key "eval/runid" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3200
  Batch size = 64


pred : <transformers.trainer_utils.EvalPrediction object at 0x7fb0ee92b070>


Trainer is attempting to log a value of "test" of type <class 'str'> for key "eval/runid" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1600, training_loss=0.379487726688385, metrics={'train_runtime': 2012.7463, 'train_samples_per_second': 12.719, 'train_steps_per_second': 0.795, 'total_flos': 253117897113600.0, 'train_loss': 0.379487726688385, 'epoch': 2.0})

In [83]:
import csv
import sys

csv.field_size_limit(sys.maxsize)


fr1 = open("/content/drive/MyDrive/collectionandqueries/all_query_ids.tsv", "r")
fr2 = open("/content/drive/MyDrive/collectionandqueries/all_queries.tsv", "r")
fr3 = open("/content/drive/MyDrive/collectionandqueries/all_document_ids.tsv", "r")
fr4 = open("/content/drive/MyDrive/collectionandqueries/all_documents.tsv", "r")
fr5 = open("/content/drive/MyDrive/collectionandqueries/all_labels.tsv", "r")

input_file1 = csv.reader(fr1, delimiter = "\t")
input_file2 = csv.reader(fr2, delimiter = "\t")
input_file3 = csv.reader(fr3, delimiter = "\t")
input_file4 = csv.reader(fr4, delimiter = "\t")
input_file5 = csv.reader(fr5, delimiter = "\t")

all_query_ids2 = []
all_queries2 = []
all_document_ids2 = []
all_documents2 = []
all_labels2 = []

for w in input_file1: 
    #print('w : ' + str(w[0]))
    all_query_ids2.append(w[0])

for w in input_file2: 
    all_queries2.append(w[0])

for w in input_file3: 
    all_document_ids2.append(w[0])

for w in input_file4: 
    all_documents2.append(w[0])

for w in input_file5: 
    all_labels2.append(w[0]) 

print('len(all_query_ids2) : ' + str(len(all_query_ids2)))
print('len(all_queries2) : ' + str(len(all_queries2)))
print('len(all_document_ids2) : ' + str(len(all_document_ids2)))
print('len(all_documents2) : ' + str(len(all_documents2)))
print('len(all_labels2) : ' + str(len(all_labels2)))

all_query_ids = []
all_queries = []
all_document_ids = []
all_documents = []
all_labels = []

length = len(all_query_ids2)
index = 0

for index in range(0, length): 
    #print('index : ' + str(index))
    all_query_ids.append(all_query_ids2[index])
    all_queries.append(all_queries2[index])
    all_document_ids.append(all_document_ids2[index])
    all_documents.append(all_documents2[index])
    all_labels.append(int(all_labels2[index]))
    """
    print('all_query_ids2[index] : ' + all_query_ids2[index])
    print('all_queries2[index] : ' + all_queries2[index])
    print('all_document_ids2[index] : ' + all_document_ids2[index])
    print('all_documents2[index] : ' + all_documents2[index])
    print('all_labels2[index] : ' + all_labels2[index])
    """

print('len(all_query_ids) : ' + str(len(all_query_ids)))
print('len(all_queries) : ' + str(len(all_queries)))
print('len(all_document_ids) : ' + str(len(all_document_ids)))
print('len(all_documents) : ' + str(len(all_documents)))
print('len(all_labels) : ' + str(len(all_labels)))

from sklearn.model_selection import train_test_split
train_queries, val_queries, train_docs, val_docs, train_labels, val_labels = train_test_split(
    all_queries, 
    all_documents, 
    all_labels, 
    test

len(all_query_ids2) : 8000
len(all_queries2) : 8000
len(all_document_ids2) : 8000
len(all_documents2) : 8000
len(all_labels2) : 8000
len(all_query_ids) : 8000
len(all_queries) : 8000
len(all_document_ids) : 8000
len(all_documents) : 8000
len(all_labels) : 8000


In [5]:
import csv
import sys

csv.field_size_limit(sys.maxsize)


fr1 = open("/content/drive/MyDrive/collectionandqueries/all_query_ids.tsv", "r")
fr2 = open("/content/drive/MyDrive/collectionandqueries/all_queries.tsv", "r")
fr3 = open("/content/drive/MyDrive/collectionandqueries/all_document_ids.tsv", "r")
fr4 = open("/content/drive/MyDrive/collectionandqueries/all_documents.tsv", "r")
fr5 = open("/content/drive/MyDrive/collectionandqueries/all_labels.tsv", "r")

input_file1 = csv.reader(fr1, delimiter = "\t")
input_file2 = csv.reader(fr2, delimiter = "\t")
input_file3 = csv.reader(fr3, delimiter = "\t")
input_file4 = csv.reader(fr4, delimiter = "\t")
input_file5 = csv.reader(fr5, delimiter = "\t")

all_query_ids2 = []
all_queries2 = []
all_document_ids2 = []
all_documents2 = []
all_labels2 = []

for w in input_file1: 
    #print('w : ' + str(w[0]))
    all_query_ids2.append(w[0])

for w in input_file2: 
    all_queries2.append(w[0])

for w in input_file3: 
    all_document_ids2.append(w[0])

for w in input_file4: 
    all_documents2.append(w[0])

for w in input_file5: 
    all_labels2.append(w[0]) 

print('len(all_query_ids2) : ' + str(len(all_query_ids2)))
print('len(all_queries2) : ' + str(len(all_queries2)))
print('len(all_document_ids2) : ' + str(len(all_document_ids2)))
print('len(all_documents2) : ' + str(len(all_documents2)))
print('len(all_labels2) : ' + str(len(all_labels2)))

all_query_ids = []
all_queries = []
all_document_ids = []
all_documents = []
all_labels = []

length = len(all_query_ids2)
index = 0

for index in range(0, length): 
    #print('index : ' + str(index))
    all_query_ids.append(all_query_ids2[index])
    all_queries.append(all_queries2[index])
    all_document_ids.append(all_document_ids2[index])
    all_documents.append(all_documents2[index])
    all_labels.append(int(all_labels2[index]))
    """
    print('all_query_ids2[index] : ' + all_query_ids2[index])
    print('all_queries2[index] : ' + all_queries2[index])
    print('all_document_ids2[index] : ' + all_document_ids2[index])
    print('all_documents2[index] : ' + all_documents2[index])
    print('all_labels2[index] : ' + all_labels2[index])
    """

print('len(all_query_ids) : ' + str(len(all_query_ids)))
print('len(all_queries) : ' + str(len(all_queries)))
print('len(all_document_ids) : ' + str(len(all_document_ids)))
print('len(all_documents) : ' + str(len(all_documents)))
print('len(all_labels) : ' + str(len(all_labels)))



len(all_query_ids2) : 8000
len(all_queries2) : 8000
len(all_document_ids2) : 8000
len(all_documents2) : 8000
len(all_labels2) : 8000
len(all_query_ids) : 8000
len(all_queries) : 8000
len(all_document_ids) : 8000
len(all_documents) : 8000
len(all_labels) : 8000


In [9]:
from sklearn.model_selection import train_test_split

print(len(all_queries))
print(len(all_documents))
train_queries, val_queries, train_docs, val_docs, train_labels, val_labels = train_test_split(
    all_queries, 
    all_documents, 
    all_labels, 
    test_size=.2
    )
print(len(train_queries))
print(len(val_queries))
print(len(train_docs))
print(len(val_docs))
print(len(train_labels))
print(len(val_labels))

8000
8000
6400
1600
6400
1600
6400
1600


In [93]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import ndcg_score
from sklearn.metrics import classification_report
import numpy as np
import evaluate
import tensorflow as tf


"""
def my_compute_metrics_ndcg(eval_pred):
    predictions, labels = eval_pred
    print ('predictions : ' + str(predictions))
    predictions2 = tf.nn.log_softmax(predictions, axis=-1)
    #print(classification_report(labels, predictions))
    return ndcg_score(predictions2, labels)
"""

def my_compute_metrics_precision(eval_pred):
    predictions, labels = eval_pred
    metric = evaluate.load("precision")
    #print('predictions : ' + str(predictions))
    #print('labels : ' + str(labels))
    #if task != "stsb":
    predictions = np.argmax(predictions, axis=-1)
    #else:
    #predictions = predictions[:]
    print(classification_report(labels, predictions))
    return metric.compute(predictions=predictions, references=labels)


# You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
# predictions and label_ids field) and has to return a dictionary string to float.
def compute_metrics2(pred):
    """
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)   
    """
    print('pred : ' + str(pred))
    # Get the metric function
    trec_eval = evaluate.load("trec_eval")
    qrel = {}
    qids, seen, i = [], {}, 0
    for q in val_queries:
      if q in seen:
        qids.append(seen[q])
      else:
        seen[q] = i
        qids.append(i)
        i+=1
    qrel={
      "query": qids,
      "q0": val_queries,
      "docid": val_docs,
      "rel": val_labels
    }
    run = {
      "query": qids,
      "q0": val_queries,
      "docid": val_docs,
      "rank": pred[1].tolist(),
      "score": [max(l) for l in pred[0].tolist()],
      "system": ["test" for i in qids]
    }    
    results = trec_eval.compute(references=[qrel], predictions=[run]) 
    return results

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=0.005,
    save_total_limit=1,              # limit the total amount of checkpoints. Deletes the older checkpoints.    
)


trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics = compute_metrics2
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 6400
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 800
  Number of trainable parameters = 1026


Epoch,Training Loss,Validation Loss,Runid,Num Ret,Num Rel,Num Rel Ret,Num Q,Map,Gm Map,Bpref,Rprec,Recip Rank,P@5,P@10,P@15,P@20,P@30,P@100,P@200,P@500,P@1000,Ndcg@5,Ndcg@10,Ndcg@15,Ndcg@20,Ndcg@30,Ndcg@100,Ndcg@200,Ndcg@500,Ndcg@1000
1,No log,0.451969,test,1600,813,813,1391,0.434220,nan,0.000000,0.434220,0.434220,0.116894,0.058447,0.038965,0.029224,0.019482,0.005845,0.002922,0.001169,0.000584,0.434220,0.434220,0.434220,0.434220,0.434220,0.434220,0.434220,0.434220,0.434220
2,0.472400,0.390789,test,1600,813,813,1391,0.434220,nan,0.000000,0.434220,0.434220,0.116894,0.058447,0.038965,0.029224,0.019482,0.005845,0.002922,0.001169,0.000584,0.434220,0.434220,0.434220,0.434220,0.434220,0.434220,0.434220,0.434220,0.434220


***** Running Evaluation *****
  Num examples = 1600
  Batch size = 64


pred : <transformers.trainer_utils.EvalPrediction object at 0x7fb0ee9ee460>


Trainer is attempting to log a value of "test" of type <class 'str'> for key "eval/runid" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-1500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 64


pred : <transformers.trainer_utils.EvalPrediction object at 0x7fb0ee945a60>


Trainer is attempting to log a value of "test" of type <class 'str'> for key "eval/runid" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=800, training_loss=0.45462855339050295, metrics={'train_runtime': 1030.8804, 'train_samples_per_second': 12.417, 'train_steps_per_second': 0.776, 'total_flos': 126558948556800.0, 'train_loss': 0.45462855339050295, 'epoch': 2.0})

In [94]:
eval = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1600
  Batch size = 64


pred : <transformers.trainer_utils.EvalPrediction object at 0x7fb0eeaf28b0>


Trainer is attempting to log a value of "test" of type <class 'str'> for key "eval/runid" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [95]:
print('eval : ' + str(eval))

eval : {'eval_loss': 0.39078933000564575, 'eval_runid': 'test', 'eval_num_ret': 1600, 'eval_num_rel': 813, 'eval_num_rel_ret': 813, 'eval_num_q': 1391, 'eval_map': 0.43421998562185476, 'eval_gm_map': nan, 'eval_bpref': 0.0, 'eval_Rprec': 0.43421998562185476, 'eval_recip_rank': 0.43421998562185476, 'eval_P@5': 0.11689432063263838, 'eval_P@10': 0.05844716031631919, 'eval_P@15': 0.0389647735442128, 'eval_P@20': 0.029223580158159596, 'eval_P@30': 0.0194823867721064, 'eval_P@100': 0.00584471603163192, 'eval_P@200': 0.00292235801581596, 'eval_P@500': 0.001168943206326384, 'eval_P@1000': 0.000584471603163192, 'eval_NDCG@5': 0.43421998562185476, 'eval_NDCG@10': 0.43421998562185476, 'eval_NDCG@15': 0.43421998562185476, 'eval_NDCG@20': 0.43421998562185476, 'eval_NDCG@30': 0.43421998562185476, 'eval_NDCG@100': 0.43421998562185476, 'eval_NDCG@200': 0.43421998562185476, 'eval_NDCG@500': 0.43421998562185476, 'eval_NDCG@1000': 0.43421998562185476, 'eval_runtime': 105.8722, 'eval_samples_per_second': 

In [96]:
trainer.predict(val_dataset)

***** Running Prediction *****
  Num examples = 1600
  Batch size = 64


pred : <transformers.trainer_utils.EvalPrediction object at 0x7fb0ee743c10>


PredictionOutput(predictions=array([[-0.248337  , -0.27784067],
       [ 0.3606272 , -0.96695346],
       [ 0.0452804 , -0.53285766],
       ...,
       [-2.5073123 ,  2.3747811 ],
       [ 0.22137338, -0.970917  ],
       [ 0.06851742, -1.2299039 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 1]), metrics={'test_loss': 0.39078933000564575, 'test_runid': 'test', 'test_num_ret': 1600, 'test_num_rel': 813, 'test_num_rel_ret': 813, 'test_num_q': 1391, 'test_map': 0.43421998562185476, 'test_gm_map': nan, 'test_bpref': 0.0, 'test_Rprec': 0.43421998562185476, 'test_recip_rank': 0.43421998562185476, 'test_P@5': 0.11689432063263838, 'test_P@10': 0.05844716031631919, 'test_P@15': 0.0389647735442128, 'test_P@20': 0.029223580158159596, 'test_P@30': 0.0194823867721064, 'test_P@100': 0.00584471603163192, 'test_P@200': 0.00292235801581596, 'test_P@500': 0.001168943206326384, 'test_P@1000': 0.000584471603163192, 'test_NDCG@5': 0.43421998562185476, 'test_NDCG@10': 0.43421998562185476, 'test_